In [1]:
import numpy as np
import pandas as pd
import math
np.set_printoptions(suppress=True)

In [2]:
user_artist_df = pd.read_csv('user_artist.csv')
test_df = pd.read_csv('test.csv')

In [3]:
new_df = (user_artist_df.copy())
new_test_df = (test_df.copy())
# mask = (new_test_df['userID'] == 145)
# mask1 = (new_test_df['userID'] == 145) | (new_test_df['userID'] == 1539) | (new_test_df['userID'] == 2214)| (new_test_df['userID'] == 211) | (new_test_df['userID'] == 3028)
# new_test_df = new_test_df[mask] 
new_test_df['weight'] = -1

In [4]:
new_test_df1 = pd.pivot_table(new_test_df, values = 'weight', index = ['userID'], columns = ['artistID'])

In [5]:
new_df1 = pd.pivot_table(new_df, values="weight", index=['userID'], columns=["artistID"])

In [6]:
new_df1.fillna(0, inplace=True)

In [7]:
# R avg 
# r_avg = (pd.DataFrame(new_df1).to_numpy()).mean()
r_avg = user_artist_df['weight'].mean()
r_avg

733.3121758765291

In [8]:
result = pd.concat([new_df1, new_test_df1])
# res1 = result.select_dtypes(np.number).max(level=0)
res2 = result.groupby(level = 0).median()

cols = sorted(list(res2.columns))
sortes3 = res2[cols]
sortes3.fillna(-2, inplace=True)

In [9]:
mtrx_R = pd.DataFrame(sortes3).to_numpy()

In [10]:
def calc_deviation_b(arr):
    n = len(arr[arr >= 0])
    if n == 0:
        return 0
    sum_arr = 0
    for i in range(len(arr)):
        if arr[i] >= 0:
            sum_arr += arr[i]
    return sum_arr/n

In [11]:
b_u = []
b_i = []
for i in range(len(mtrx_R)):
    b_u.append((calc_deviation_b(mtrx_R[i]) - r_avg))

mtrx_R_tr = mtrx_R.T

for i in range(len(mtrx_R_tr)):
    b_i.append((calc_deviation_b(mtrx_R_tr[i]) - r_avg))

In [12]:
# create R_hat
to_filled = []
mtrx_R_hat = np.zeros((len(mtrx_R), len(mtrx_R[0])))
for i in range(len(mtrx_R)):
     for j in range(len(mtrx_R[0])):
        # here don't use -1 yet.
        if mtrx_R[i,j] == -2:
            mtrx_R_hat[i,j] = 0
        else:
            mtrx_R_hat[i,j] = r_avg + b_u[i] + b_i[j]
            if mtrx_R[i,j] == -1:
                to_filled.append((i,j))
        
            

In [13]:
# create R_tilda
mtrx_R_tilda = mtrx_R - mtrx_R_hat

# put -1 after the sub.

for (i,j) in to_filled:
    mtrx_R_tilda[i,j] = -1


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
def special_inner_profuct(mtrx): # need a faster way to calculate- 
    mtrx_transpose = mtrx.T
    D = cosine_similarity(mtrx_transpose)
#     mtrx_R_transpose_copy = mtrx_transpose.copy()
#     mtrx_R_transpose_copy = np.where(mtrx_R_transpose_copy == -1, 0, mtrx_R_transpose_copy)
#     mtrx_R_transpose_copy = np.where(mtrx_R_transpose_copy == -2, 0, mtrx_R_transpose_copy)
#     D = np.zeros((len(mtrx[0]), len(mtrx[0])))
    
#     for i in range(len(mtrx_R_transpose_copy)):
#         for j in range(len(mtrx_R_transpose_copy)):
#             mone = np.dot(mtrx_R_transpose_copy[i], mtrx_R_transpose_copy[j])
#             x = 0
#             y = 0
#             for k in range(len(mtrx_R_transpose_copy[0])):
#                 # run n^3 where n = 16548 is the number of movies.
                
#                 if mtrx_R_transpose_copy[i][k] != -1 and mtrx_R_transpose_copy[j][k] != -1 and mtrx_R_transpose_copy[i][k] != -2 and mtrx_R_transpose_copy[j][k] != -2:
#                     x += (mtrx_R_transpose_copy[i][k] ** 2)
#                     y += (mtrx_R_transpose_copy[j][k] ** 2)     
#             norm1 = x ** 0.5
#             norm2 = y ** 0.5
            
#             mechane = norm1 * norm2
#             if mechane == 0:
#                 D[i,j] = 0
#             else:
#                 D[i,j] = mone / mechane
    
#     for i in range(len(mtrx)):
#         print(mtrx[i])
#     return
            
    return D

In [15]:
def get_nearest(similarities, n):
    # n is the number of neighboors we want.
    
    sim_arr = []
    
    for i in range(len(similarities)):
        array = similarities[i].copy()
        array_non_neg = array.copy()
        for j in range(len(array)):
            if int(array_non_neg[j]) == 1 or int(array_non_neg[j]) == -1:
                array_non_neg[j] = 0
            if array_non_neg[j] < 0:
                array_non_neg[j] = array_non_neg[j] * (-1)

        ind = np.argpartition(array_non_neg, -n)[-n:]
        
        top2 = array[sorted(ind)]
        sim_arr.append((top2, sorted(ind)))

    
    return sim_arr

In [16]:
def get_final_R(mtrx_R_hat, mtrx_R_tilda, similiraties):
    print("in get_final_R")
    # nearest = get_nearest(similarities, len(similarities)) # get the 2 nearest movies and their indexes.
    print("found nearest")
    
    mtrx_R_final = np.zeros((len(mtrx_R_hat), len(mtrx_R_hat[0])))
    
    print(f"need to run {len(mtrx_R_hat)} times")
    for i in range(len(mtrx_R_hat)):
        most_similar_mvs_to_j = similiraties[i]
        r_tilda  = mtrx_R_tilda[i]
        mone1 = np.dot(most_similar_mvs_to_j, r_tilda)
        mechane1 = 0
        for k in range(len(similiraties[i])):
            mechane1 += abs(similiraties[i][k])
            if mechane1 == 0:
                mtrx_R_final[i,j] = mtrx_R_hat[i,j]
            else:
                mtrx_R_final[i,j] = mtrx_R_hat[i,j] + mone1/mechane1
        print(f"{i} iteration done")
       # for j in range(len(mtrx_R_hat[0])):
#             if (i,j) != (0,1) and (i,j) != (1,0): # this will be replaced by the indexes we extract (cells which need to be filled)
#                 mtrx_R_final[i,j] = mtrx_R_hat[i,j] # only predict for the missing ones.
#                 continue
            
#             most_similar_mvs_to_j = nearest[j][1] # similarity indexes.
#             r_tilda  = (mtrx_R_tilda[i][most_similar_mvs_to_j])
#             mone1 = np.dot(nearest[j][0], r_tilda)
            
            
            
            
#             mechane1 = 0
#             for k in range(len(nearest[j][0])):
#                 mechane1 += abs(nearest[j][0][k])
                
#             if mechane1 == 0:
#                 mtrx_R_final[i,j] = mtrx_R_hat[i,j]
#             else:
#                 mtrx_R_final[i,j] = mtrx_R_hat[i,j] + mone1/mechane1
#         print(f"{i} iteration done")
    return mtrx_R_final

In [17]:
similarities = special_inner_profuct(mtrx_R_tilda)
print("found simliraties matrix")
m = get_final_R(mtrx_R_hat, mtrx_R_tilda, similarities)
m

found simliraties matrix
in get_final_R
found nearest
need to run 1892 times
0 iteration done
1 iteration done
2 iteration done
3 iteration done
4 iteration done
5 iteration done
6 iteration done
7 iteration done
8 iteration done
9 iteration done
10 iteration done
11 iteration done
12 iteration done
13 iteration done
14 iteration done
15 iteration done
16 iteration done
17 iteration done
18 iteration done
19 iteration done
20 iteration done
21 iteration done
22 iteration done
23 iteration done
24 iteration done
25 iteration done
26 iteration done
27 iteration done
28 iteration done
29 iteration done
30 iteration done
31 iteration done
32 iteration done
33 iteration done
34 iteration done
35 iteration done
36 iteration done
37 iteration done
38 iteration done
39 iteration done
40 iteration done
41 iteration done
42 iteration done
43 iteration done
44 iteration done
45 iteration done
46 iteration done
47 iteration done
48 iteration done
49 iteration done
50 iteration done
51 iteration do

435 iteration done
436 iteration done
437 iteration done
438 iteration done
439 iteration done
440 iteration done
441 iteration done
442 iteration done
443 iteration done
444 iteration done
445 iteration done
446 iteration done
447 iteration done
448 iteration done
449 iteration done
450 iteration done
451 iteration done
452 iteration done
453 iteration done
454 iteration done
455 iteration done
456 iteration done
457 iteration done
458 iteration done
459 iteration done
460 iteration done
461 iteration done
462 iteration done
463 iteration done
464 iteration done
465 iteration done
466 iteration done
467 iteration done
468 iteration done
469 iteration done
470 iteration done
471 iteration done
472 iteration done
473 iteration done
474 iteration done
475 iteration done
476 iteration done
477 iteration done
478 iteration done
479 iteration done
480 iteration done
481 iteration done
482 iteration done
483 iteration done
484 iteration done
485 iteration done
486 iteration done
487 iteratio

868 iteration done
869 iteration done
870 iteration done
871 iteration done
872 iteration done
873 iteration done
874 iteration done
875 iteration done
876 iteration done
877 iteration done
878 iteration done
879 iteration done
880 iteration done
881 iteration done
882 iteration done
883 iteration done
884 iteration done
885 iteration done
886 iteration done
887 iteration done
888 iteration done
889 iteration done
890 iteration done
891 iteration done
892 iteration done
893 iteration done
894 iteration done
895 iteration done
896 iteration done
897 iteration done
898 iteration done
899 iteration done
900 iteration done
901 iteration done
902 iteration done
903 iteration done
904 iteration done
905 iteration done
906 iteration done
907 iteration done
908 iteration done
909 iteration done
910 iteration done
911 iteration done
912 iteration done
913 iteration done
914 iteration done
915 iteration done
916 iteration done
917 iteration done
918 iteration done
919 iteration done
920 iteratio

1288 iteration done
1289 iteration done
1290 iteration done
1291 iteration done
1292 iteration done
1293 iteration done
1294 iteration done
1295 iteration done
1296 iteration done
1297 iteration done
1298 iteration done
1299 iteration done
1300 iteration done
1301 iteration done
1302 iteration done
1303 iteration done
1304 iteration done
1305 iteration done
1306 iteration done
1307 iteration done
1308 iteration done
1309 iteration done
1310 iteration done
1311 iteration done
1312 iteration done
1313 iteration done
1314 iteration done
1315 iteration done
1316 iteration done
1317 iteration done
1318 iteration done
1319 iteration done
1320 iteration done
1321 iteration done
1322 iteration done
1323 iteration done
1324 iteration done
1325 iteration done
1326 iteration done
1327 iteration done
1328 iteration done
1329 iteration done
1330 iteration done
1331 iteration done
1332 iteration done
1333 iteration done
1334 iteration done
1335 iteration done
1336 iteration done
1337 iteration done


1698 iteration done
1699 iteration done
1700 iteration done
1701 iteration done
1702 iteration done
1703 iteration done
1704 iteration done
1705 iteration done
1706 iteration done
1707 iteration done
1708 iteration done
1709 iteration done
1710 iteration done
1711 iteration done
1712 iteration done
1713 iteration done
1714 iteration done
1715 iteration done
1716 iteration done
1717 iteration done
1718 iteration done
1719 iteration done
1720 iteration done
1721 iteration done
1722 iteration done
1723 iteration done
1724 iteration done
1725 iteration done
1726 iteration done
1727 iteration done
1728 iteration done
1729 iteration done
1730 iteration done
1731 iteration done
1732 iteration done
1733 iteration done
1734 iteration done
1735 iteration done
1736 iteration done
1737 iteration done
1738 iteration done
1739 iteration done
1740 iteration done
1741 iteration done
1742 iteration done
1743 iteration done
1744 iteration done
1745 iteration done
1746 iteration done
1747 iteration done


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
error = 0
for i in range(len(m)):
    print(f"iteration {i} out of {len(m)}")
    for j in range(len(m[0])):
        if mtrx_R[i][j] <= 0:
            continue
#         if mtrx_R[i][j] == 0:
#             mtrx_R[i][j] = 1
#            m[i][j] = 1
        if m[i][j] <= 0:
            m[i][j] = 1
        error += ((math.log(m[i][j]) - math.log(mtrx_R[i][j])) ** 2)
    print(error)
        
print(error)        

iteration 0 out of 1892
1138.4700881036517
iteration 1 out of 1892
2254.653533269316
iteration 2 out of 1892
4327.310028304106
iteration 3 out of 1892
4503.541922029155
iteration 4 out of 1892
6270.5570507585035
iteration 5 out of 1892
7050.776185947687
iteration 6 out of 1892
8080.515883231114
iteration 7 out of 1892
9579.858886231703
iteration 8 out of 1892
11016.187088055349
iteration 9 out of 1892
12055.672198304825
iteration 10 out of 1892
12055.672198304825
iteration 11 out of 1892
13997.332085803271
iteration 12 out of 1892
15484.830120748342
iteration 13 out of 1892
17080.008804742618
iteration 14 out of 1892
18656.63984385171
iteration 15 out of 1892
19687.0983253621
iteration 16 out of 1892
21562.960107367795
iteration 17 out of 1892
22938.08778664916
iteration 18 out of 1892
24422.593421819365
iteration 19 out of 1892
25109.48402550691
iteration 20 out of 1892
26937.997645922398
iteration 21 out of 1892
27982.671874440523
iteration 22 out of 1892
28706.541198444327
iteration

245707.33256489522
iteration 188 out of 1892
247057.89473910313
iteration 189 out of 1892
248350.3858927174
iteration 190 out of 1892
248860.50186312548
iteration 191 out of 1892
250154.5902981267
iteration 192 out of 1892
251420.6062929444
iteration 193 out of 1892
252263.39553747652
iteration 194 out of 1892
253796.3941443617
iteration 195 out of 1892
255501.00022542448
iteration 196 out of 1892
257187.1944416932
iteration 197 out of 1892
259236.84796937107
iteration 198 out of 1892
260951.72966249142
iteration 199 out of 1892
261954.4698219059
iteration 200 out of 1892
264425.5565727611
iteration 201 out of 1892
265267.4729743737
iteration 202 out of 1892
266137.78583478555
iteration 203 out of 1892
268005.24113644334
iteration 204 out of 1892
269725.9243571824
iteration 205 out of 1892
271056.5907691436
iteration 206 out of 1892
271295.44251776696
iteration 207 out of 1892
273399.71500571154
iteration 208 out of 1892
274851.5914109626
iteration 209 out of 1892
276501.39979083056
it

502510.12543997896
iteration 373 out of 1892
503881.7872969964
iteration 374 out of 1892
505586.3104574516
iteration 375 out of 1892
507404.0374375337
iteration 376 out of 1892
508583.16555039317
iteration 377 out of 1892
510127.6265975236
iteration 378 out of 1892
512206.5759498041
iteration 379 out of 1892
513501.10268832085
iteration 380 out of 1892
514595.79431435734
iteration 381 out of 1892
516449.363812003
iteration 382 out of 1892
518113.33211427636
iteration 383 out of 1892
518890.3657569988
iteration 384 out of 1892
519950.76408346964
iteration 385 out of 1892
522086.272348812
iteration 386 out of 1892
524783.6039996006
iteration 387 out of 1892
525076.6143929275
iteration 388 out of 1892
525815.0284374895
iteration 389 out of 1892
527898.5049477962
iteration 390 out of 1892
527974.8143480668
iteration 391 out of 1892
529630.3181132072
iteration 392 out of 1892
531571.7264385687
iteration 393 out of 1892
533319.0844084185
iteration 394 out of 1892
533423.5267611785
iteration 

782584.9143459422
iteration 560 out of 1892
783463.4055799377
iteration 561 out of 1892
785321.1318362687
iteration 562 out of 1892
786572.0265844896
iteration 563 out of 1892
787910.7751567207
iteration 564 out of 1892
788987.6565411563
iteration 565 out of 1892
789793.3648018765
iteration 566 out of 1892
791453.2212242947
iteration 567 out of 1892
791888.5009959426
iteration 568 out of 1892
791890.1883979173
iteration 569 out of 1892
793158.9137316126
iteration 570 out of 1892
793234.4813280539
iteration 571 out of 1892
794583.0379432098
iteration 572 out of 1892
795748.0979100467
iteration 573 out of 1892
797341.7558853949
iteration 574 out of 1892
798229.7231111927
iteration 575 out of 1892
799213.7803006344
iteration 576 out of 1892
800897.7194685494
iteration 577 out of 1892
802027.9262491148
iteration 578 out of 1892
802565.7791205716
iteration 579 out of 1892
803636.7925972369
iteration 580 out of 1892
804873.2323791011
iteration 581 out of 1892
805831.6547523479
iteration 582 

1033853.9810146337
iteration 748 out of 1892
1034520.416907348
iteration 749 out of 1892
1037233.5384407683
iteration 750 out of 1892
1038881.8467325885
iteration 751 out of 1892
1040902.5428385182
iteration 752 out of 1892
1042382.3184917015
iteration 753 out of 1892
1043203.1129229917
iteration 754 out of 1892
1045012.8858909339
iteration 755 out of 1892
1046199.4005914197
iteration 756 out of 1892
1047945.1512456031
iteration 757 out of 1892
1049460.8789157618
iteration 758 out of 1892
1050405.584336085
iteration 759 out of 1892
1052062.4622185677
iteration 760 out of 1892
1053474.1466883004
iteration 761 out of 1892
1054776.8683281718
iteration 762 out of 1892
1056242.5191054258
iteration 763 out of 1892
1057241.9564474705
iteration 764 out of 1892
1058604.7935282164
iteration 765 out of 1892
1059533.2891222588
iteration 766 out of 1892
1061388.4509205327
iteration 767 out of 1892
1062900.0168036018
iteration 768 out of 1892
1064817.940776217
iteration 769 out of 1892
1066283.73325

...

1526302.662930642
iteration 1114 out of 1892
1527297.7599898323
iteration 1115 out of 1892
1529384.0619516554
iteration 1116 out of 1892
1531546.8505013536
iteration 1117 out of 1892
1533156.6090495498
iteration 1118 out of 1892
1534429.5055984526
iteration 1119 out of 1892
1535733.4096466498
iteration 1120 out of 1892
1537031.5071341724
iteration 1121 out of 1892
1538463.7670529245
iteration 1122 out of 1892
1540021.9342868957
iteration 1123 out of 1892
1541848.7689324345
iteration 1124 out of 1892
1542830.7715545732
iteration 1125 out of 1892
1543999.7827549013
iteration 1126 out of 1892
1545609.6698985314
iteration 1127 out of 1892
1545915.950870664
iteration 1128 out of 1892
1547207.236796725
iteration 1129 out of 1892
1548186.7794898478
iteration 1130 out of 1892
1549809.732173667
iteration 1131 out of 1892
1550772.9730625644
iteration 1132 out of 1892
1552308.9133494135
iteration 1133 out of 1892
1553626.1627778157
iteration 1134 out of 1892
1555274.512461434
iteration 1135 out o

1777820.9932891072
iteration 1295 out of 1892
1779181.8204317244
iteration 1296 out of 1892
1779972.6072959371
iteration 1297 out of 1892
1781696.0680546875
iteration 1298 out of 1892
1782796.5437292154
iteration 1299 out of 1892
1783752.3030637854
iteration 1300 out of 1892
1785068.0153855884
iteration 1301 out of 1892
1785707.1998190908
iteration 1302 out of 1892
1788068.3676842814
iteration 1303 out of 1892
1789812.271866726
iteration 1304 out of 1892
1791244.5780612568
iteration 1305 out of 1892
1792300.531410789
iteration 1306 out of 1892
1793609.0710120806
iteration 1307 out of 1892
1794608.425777992
iteration 1308 out of 1892
1795885.7565811838
iteration 1309 out of 1892
1796745.321689441
iteration 1310 out of 1892
1797733.928328975
iteration 1311 out of 1892
1798618.3428031085
iteration 1312 out of 1892
1800304.305494788
iteration 1313 out of 1892
1801506.730226404
iteration 1314 out of 1892
1803530.2907122879
iteration 1315 out of 1892
1804914.5738953983
iteration 1316 out of 

2026226.8099006533
iteration 1476 out of 1892
2027376.0671822145
iteration 1477 out of 1892
2028627.4356771486
iteration 1478 out of 1892
2030308.8348256818
iteration 1479 out of 1892
2031755.1006315018
iteration 1480 out of 1892
2033577.2880464147
iteration 1481 out of 1892
2034942.66906181
iteration 1482 out of 1892
2034944.5908738656
iteration 1483 out of 1892
2036148.0878669594
iteration 1484 out of 1892
2037335.4520757431
iteration 1485 out of 1892
2038965.3809572628
iteration 1486 out of 1892
2040336.7172114956
iteration 1487 out of 1892
2041435.6330771137
iteration 1488 out of 1892
2042709.8707227365
iteration 1489 out of 1892
2044589.9331838787
iteration 1490 out of 1892
2046322.0494108547
iteration 1491 out of 1892
2048133.7468416505
iteration 1492 out of 1892
2049479.357309985
iteration 1493 out of 1892
2050788.5871378134
iteration 1494 out of 1892
2052048.715043805
iteration 1495 out of 1892
2053927.6664745812
iteration 1496 out of 1892
2055219.189840128
iteration 1497 out o

2273876.349328025
iteration 1657 out of 1892
2275777.886517452
iteration 1658 out of 1892
2277307.1449203496
iteration 1659 out of 1892
2279151.6039115507
iteration 1660 out of 1892
2281308.99450791
iteration 1661 out of 1892
2281312.6037219404
iteration 1662 out of 1892
2281921.906788557
iteration 1663 out of 1892
2283426.1054560076
iteration 1664 out of 1892
2285447.5864187772
iteration 1665 out of 1892
2286503.7177599636
iteration 1666 out of 1892
2288192.11680673
iteration 1667 out of 1892
2288848.0907585924
iteration 1668 out of 1892
2289932.1303844056
iteration 1669 out of 1892
2291634.6976128016
iteration 1670 out of 1892
2293071.5994533394
iteration 1671 out of 1892
2294994.082746146
iteration 1672 out of 1892
2296451.9731068476
iteration 1673 out of 1892
2297476.210501626
iteration 1674 out of 1892
2299890.148192451
iteration 1675 out of 1892
2301571.403128345
iteration 1676 out of 1892
2302119.6252979646
iteration 1677 out of 1892
2302721.59538954
iteration 1678 out of 1892
2

2516967.932425042
iteration 1839 out of 1892
2518646.397442052
iteration 1840 out of 1892
2520693.488087205
iteration 1841 out of 1892
2522096.6490183226
iteration 1842 out of 1892
2523898.6654813564
iteration 1843 out of 1892
2524772.0622811937
iteration 1844 out of 1892
2526603.0275141774
iteration 1845 out of 1892
2528600.583331137
iteration 1846 out of 1892
2530451.4585056845
iteration 1847 out of 1892
2531969.1852355506
iteration 1848 out of 1892
2533366.955065187
iteration 1849 out of 1892
2535593.7138339025
iteration 1850 out of 1892
2537694.0197248305
iteration 1851 out of 1892
2538864.3556669676
iteration 1852 out of 1892
2540181.838580607
iteration 1853 out of 1892
2542329.577697431
iteration 1854 out of 1892
2543426.665417137
iteration 1855 out of 1892
2544356.3959502
iteration 1856 out of 1892
2545463.7493085633
iteration 1857 out of 1892
2546593.7200620794
iteration 1858 out of 1892
2548287.8623467083
iteration 1859 out of 1892
2549804.908858759
iteration 1860 out of 1892


In [19]:
file = open("out.txt", "w")
str_mat = repr(m)
file.write("final matrix = " + m + "\n")
file.close()

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U15'), dtype('float64')) -> None

In [22]:
mtrx_test = np.zeros((len(mtrx_R), len(mtrx_R[0])))
for i in range(len(mtrx_R)):
    for j in range(len(mtrx_R[0])):
        mtrx_test[i][j] = r_avg


In [23]:
mtrx_test

array([[733.31217588, 733.31217588, 733.31217588, ..., 733.31217588,
        733.31217588, 733.31217588],
       [733.31217588, 733.31217588, 733.31217588, ..., 733.31217588,
        733.31217588, 733.31217588],
       [733.31217588, 733.31217588, 733.31217588, ..., 733.31217588,
        733.31217588, 733.31217588],
       ...,
       [733.31217588, 733.31217588, 733.31217588, ..., 733.31217588,
        733.31217588, 733.31217588],
       [733.31217588, 733.31217588, 733.31217588, ..., 733.31217588,
        733.31217588, 733.31217588],
       [733.31217588, 733.31217588, 733.31217588, ..., 733.31217588,
        733.31217588, 733.31217588]])

In [24]:
error_test = 0
for i in range(len(m)):
    print(f"iteration {i} out of {len(m)}")
    for j in range(len(m[0])):
        if mtrx_R[i][j] < 0:
            continue
        if mtrx_R[i][j] == 0:
            mtrx_R[i][j] = 1
            m[i][j] = 1
        if m[i][j] <= 0:
            m[i][j] = 1
        error_test += ((math.log(m[i][j]) - math.log(mtrx_test[i][j])) ** 2)
    print(error_test)
        
print(error_test)  

iteration 0 out of 1892
719429.9524244221
iteration 1 out of 1892
1438903.432798542
iteration 2 out of 1892
2158289.857272961
iteration 3 out of 1892
2877763.3376432327
iteration 4 out of 1892
3597149.762114131
iteration 5 out of 1892
4316536.186586337
iteration 6 out of 1892
5035792.02721587
iteration 7 out of 1892
5755178.451694464
iteration 8 out of 1892
6474564.8761730585
iteration 9 out of 1892
7193994.82860134
iteration 10 out of 1892
7913468.308979308
iteration 11 out of 1892
8632941.789357277
iteration 12 out of 1892
9352371.741785558
iteration 13 out of 1892
10071845.222163526
iteration 14 out of 1892
10791275.174591808
iteration 15 out of 1892
11510705.127020089
iteration 16 out of 1892
12230048.023548996
iteration 17 out of 1892
12949347.392128216
iteration 18 out of 1892
13668777.344556497
iteration 19 out of 1892
14388120.241085405
iteration 20 out of 1892
15107550.193513686
iteration 21 out of 1892
15826849.562092906
iteration 22 out of 1892
16546223.781970132
iteration 2

135248843.50627315
iteration 188 out of 1892
135968316.9867127
iteration 189 out of 1892
136687703.41125286
iteration 190 out of 1892
137407176.8916924
iteration 191 out of 1892
138126519.78828287
iteration 192 out of 1892
138845862.68487334
iteration 193 out of 1892
139565249.1094135
iteration 194 out of 1892
140284592.00600398
iteration 195 out of 1892
141003978.43054414
iteration 196 out of 1892
141723321.3271346
iteration 197 out of 1892
142442707.75167477
iteration 198 out of 1892
143162050.64826524
iteration 199 out of 1892
143881524.1287048
iteration 200 out of 1892
144600910.55324495
iteration 201 out of 1892
145320253.44983542
iteration 202 out of 1892
146039683.40232527
iteration 203 out of 1892
146759069.82686543
iteration 204 out of 1892
147478499.7793553
iteration 205 out of 1892
148197929.73184514
iteration 206 out of 1892
148917316.1563853
iteration 207 out of 1892
149636659.05297577
iteration 208 out of 1892
150356001.94956625
iteration 209 out of 1892
151075475.4300058

268340269.38615733
iteration 373 out of 1892
269059655.8106975
iteration 374 out of 1892
269779042.23523766
iteration 375 out of 1892
270498428.6597778
iteration 376 out of 1892
271217684.5004689
iteration 377 out of 1892
271937157.98090845
iteration 378 out of 1892
272656500.8774989
iteration 379 out of 1892
273375800.2461397
iteration 380 out of 1892
274095273.72657925
iteration 381 out of 1892
274814703.6790691
iteration 382 out of 1892
275534046.5756596
iteration 383 out of 1892
276253433.00019974
iteration 384 out of 1892
276972732.3688405
iteration 385 out of 1892
277692205.84928006
iteration 386 out of 1892
278411635.8017699
iteration 387 out of 1892
279131022.2263101
iteration 388 out of 1892
279850452.1787999
iteration 389 out of 1892
280569838.6033401
iteration 390 out of 1892
281289312.08377963
iteration 391 out of 1892
282008742.0362695
iteration 392 out of 1892
282728171.98875934
iteration 393 out of 1892
283447514.8853498
iteration 394 out of 1892
284166944.83783966
itera

402149891.76665574
iteration 559 out of 1892
402869321.7191456
iteration 560 out of 1892
403588795.19958514
iteration 561 out of 1892
404308268.6800247
iteration 562 out of 1892
405027638.53479356
iteration 563 out of 1892
405746937.90343434
iteration 564 out of 1892
406466324.3279745
iteration 565 out of 1892
407185623.6966153
iteration 566 out of 1892
407904966.59320575
iteration 567 out of 1892
408624309.4897962
iteration 568 out of 1892
409343782.97023576
iteration 569 out of 1892
410063038.81092685
iteration 570 out of 1892
410782512.2913664
iteration 571 out of 1892
411501942.24385625
iteration 572 out of 1892
412221285.1404467
iteration 573 out of 1892
412940453.92523843
iteration 574 out of 1892
413659883.8777283
iteration 575 out of 1892
414379357.3581678
iteration 576 out of 1892
415098830.8386074
iteration 577 out of 1892
415818304.3190469
iteration 578 out of 1892
416537777.79948646
iteration 579 out of 1892
417257033.64017755
iteration 580 out of 1892
417976507.1206171
ite

535240193.56545544
iteration 744 out of 1892
535959536.4620459
iteration 745 out of 1892
536678922.88658607
iteration 746 out of 1892
537398309.3111262
iteration 747 out of 1892
538117739.2636161
iteration 748 out of 1892
538837038.6322569
iteration 749 out of 1892
539556468.5847467
iteration 750 out of 1892
540275898.5372366
iteration 751 out of 1892
540995241.433827
iteration 752 out of 1892
541714627.8583672
iteration 753 out of 1892
542434101.3388067
iteration 754 out of 1892
543153444.2353972
iteration 755 out of 1892
543872830.6599374
iteration 756 out of 1892
544592042.9726788
iteration 757 out of 1892
545311429.397219
iteration 758 out of 1892
546030815.8217591
iteration 759 out of 1892
546750245.774249
iteration 760 out of 1892
547469719.2546885
iteration 761 out of 1892
548189105.6792287
iteration 762 out of 1892
548908448.5758191
iteration 763 out of 1892
549627878.528309
iteration 764 out of 1892
550347352.0087485
iteration 765 out of 1892
551066738.4332887
iteration 766 ou

...

805017749.9601831
iteration 1119 out of 1892
805737179.912673
iteration 1120 out of 1892
806456522.8092635
iteration 1121 out of 1892
807175996.289703
iteration 1122 out of 1892
807895295.6583438
iteration 1123 out of 1892
808614682.082884
iteration 1124 out of 1892
809334068.5074241
iteration 1125 out of 1892
810053411.4040146
iteration 1126 out of 1892
810772841.3565044
iteration 1127 out of 1892
811492314.836944
iteration 1128 out of 1892
812211788.3173835
iteration 1129 out of 1892
812931218.2698734
iteration 1130 out of 1892
813650648.2223632
iteration 1131 out of 1892
814370121.7028028
iteration 1132 out of 1892
815089421.0714436
iteration 1133 out of 1892
815808894.5518831
iteration 1134 out of 1892
816528368.0323226
iteration 1135 out of 1892
817247841.5127622
iteration 1136 out of 1892
817967227.9373024
iteration 1137 out of 1892
818686614.3618425
iteration 1138 out of 1892
819406044.3143324
iteration 1139 out of 1892
820125474.2668222
iteration 1140 out of 1892
820844817.1634

938109113.6304785
iteration 1304 out of 1892
938828456.527069
iteration 1305 out of 1892
939547842.9516091
iteration 1306 out of 1892
940267272.904099
iteration 1307 out of 1892
940986615.8006895
iteration 1308 out of 1892
941705958.6972799
iteration 1309 out of 1892
942425388.6497698
iteration 1310 out of 1892
943144862.1302093
iteration 1311 out of 1892
943864335.6106489
iteration 1312 out of 1892
944583765.5631387
iteration 1313 out of 1892
945303108.4597292
iteration 1314 out of 1892
946022494.8842694
iteration 1315 out of 1892
946741924.8367592
iteration 1316 out of 1892
947461311.2612994
iteration 1317 out of 1892
948180610.6299402
iteration 1318 out of 1892
948899953.5265306
iteration 1319 out of 1892
949619383.4790205
iteration 1320 out of 1892
950338813.4315103
iteration 1321 out of 1892
951058199.8560505
iteration 1322 out of 1892
951777673.33649
iteration 1323 out of 1892
952497059.7610302
iteration 1324 out of 1892
953216446.1855704
iteration 1325 out of 1892
953935832.6101

1069040454.7786007
iteration 1486 out of 1892
1069759797.6751912
iteration 1487 out of 1892
1070479053.5158823
iteration 1488 out of 1892
1071198483.4683721
iteration 1489 out of 1892
1071917913.420862
iteration 1490 out of 1892
1072637256.3174524
iteration 1491 out of 1892
1073356686.2699423
iteration 1492 out of 1892
1074076159.7494662
iteration 1493 out of 1892
1074795589.6999857
iteration 1494 out of 1892
1075514976.1225557
iteration 1495 out of 1892
1076234406.0730753
iteration 1496 out of 1892
1076953836.0235949
iteration 1497 out of 1892
1077673309.502064
iteration 1498 out of 1892
1078392739.4525836
iteration 1499 out of 1892
1079112125.8751535
iteration 1500 out of 1892
1079831555.825673
iteration 1501 out of 1892
1080550898.7202935
iteration 1502 out of 1892
1081270241.614914
iteration 1503 out of 1892
1081989715.093383
iteration 1504 out of 1892
1082709101.515953
iteration 1505 out of 1892
1083428531.4664726
iteration 1506 out of 1892
1084147961.4169922
iteration 1507 out of

1199253388.045212
iteration 1667 out of 1892
1199972643.8839333
iteration 1668 out of 1892
1200692117.3624024
iteration 1669 out of 1892
1201411590.8408716
iteration 1670 out of 1892
1202130890.2075424
iteration 1671 out of 1892
1202850276.6301124
iteration 1672 out of 1892
1203569706.580632
iteration 1673 out of 1892
1204289093.003202
iteration 1674 out of 1892
1205008566.481671
iteration 1675 out of 1892
1205727952.904241
iteration 1676 out of 1892
1206447382.8547606
iteration 1677 out of 1892
1207166769.2773306
iteration 1678 out of 1892
1207886199.2278502
iteration 1679 out of 1892
1208605585.6504202
iteration 1680 out of 1892
1209324972.0729902
iteration 1681 out of 1892
1210044271.439661
iteration 1682 out of 1892
1210763701.3901806
iteration 1683 out of 1892
1211483131.3407001
iteration 1684 out of 1892
1212202430.707371
iteration 1685 out of 1892
1212921817.129941
iteration 1686 out of 1892
1213641203.552511
iteration 1687 out of 1892
1214360633.5030305
iteration 1688 out of 18

1329464534.5532765
iteration 1849 out of 1892
1330184008.0317457
iteration 1850 out of 1892
1330903350.926366
iteration 1851 out of 1892
1331622780.8768857
iteration 1852 out of 1892
1332342123.771506
iteration 1853 out of 1892
1333061249.0263786
iteration 1854 out of 1892
1333780591.920999
iteration 1855 out of 1892
1334500021.8715186
iteration 1856 out of 1892
1335219364.766139
iteration 1857 out of 1892
1335938707.6607594
iteration 1858 out of 1892
1336658181.1392286
iteration 1859 out of 1892
1337377567.5617986
iteration 1860 out of 1892
1338096910.456419
iteration 1861 out of 1892
1338816296.878989
iteration 1862 out of 1892
1339535683.301559
iteration 1863 out of 1892
1340255113.2520785
iteration 1864 out of 1892
1340974325.5628502
iteration 1865 out of 1892
1341693799.0413194
iteration 1866 out of 1892
1342413272.5197885
iteration 1867 out of 1892
1343132658.9423585
iteration 1868 out of 1892
1343852088.892878
iteration 1869 out of 1892
1344571562.3713472
iteration 1870 out of 1